In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install xlrd


DEPRECATION: beakerx-base 2.2.0 has a non-standard dependency specifier ipywidgets<8pandas,>=7.5.1. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of beakerx-base or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import xlrd
print(f"xlrd version: {xlrd.__version__}")


xlrd version: 2.0.1


In [4]:
# Define the path to your Excel file
file_path = 'Annual change in the number of residences by type of activity and by municipality size class.xls'

# Load the Excel file
excel_file = pd.ExcelFile(file_path, engine='xlrd')

# Retrieve the list of sheet names
sheet_names = excel_file.sheet_names
print(f"Available Sheets: {sheet_names}")

# Dictionary to store cleaned DataFrames keyed by sheet name
cleaned_data = {}

def combine_header_rows(df, header_start=0, header_end=9, join_separator=''):
    """
    Combines multiple header rows into a single header by concatenating the text.

    Parameters:
    - df: pandas DataFrame read without headers.
    - header_start: Starting row index for headers (inclusive).
    - header_end: Ending row index for headers (exclusive).
    - join_separator: Separator to use when joining header strings.

    Returns:
    - df_cleaned: DataFrame with combined headers and header rows removed.
    - combined_headers: List of combined header strings.
    """
    # Extract header rows
    header_data = df.iloc[header_start:header_end].fillna('')  # Replace NaN with empty strings
    
    # Combine header rows for each column by concatenating non-empty strings
    combined_headers = header_data.apply(
        lambda x: join_separator.join([str(item).strip() for item in x if str(item).strip() != '']),
        axis=0
    )
    
    # Assign combined headers to the DataFrame
    df.columns = combined_headers
    
    # Remove the header rows from the DataFrame
    df_cleaned = df.iloc[header_end:].reset_index(drop=True)
    
    return df_cleaned, combined_headers.tolist()


# Iterate over each sheet
for sheet in sheet_names:
    print(f"\nProcessing Sheet: {sheet}")
    
    # Read the sheet without headers
    df = pd.read_excel(file_path, sheet_name=sheet, header=None, engine='xlrd')
    
    # Combine header rows (rows 0 through 8) into a single header
    df_cleaned, combined_headers = combine_header_rows(df, header_start=0, header_end=9, join_separator='')
    
    # Display combined headers for verification
    print("Combined Headers:")
    for idx, header in enumerate(combined_headers):
        print(f"Column {idx}: '{header}'")
    
    # Remove columns where the combined header is empty
    # This handles any columns that were entirely empty in header rows
    non_empty_columns = [col for col in df_cleaned.columns if col.strip() != '']
    df_cleaned = df_cleaned[non_empty_columns]
    
    # Reset index after removing columns
    df_cleaned.reset_index(drop=True, inplace=True)
    
    # Drop any rows that are completely NaN
    df_cleaned.dropna(how='all', inplace=True)
    
    # Reset index after dropping rows
    df_cleaned.reset_index(drop=True, inplace=True)
    
    # Remove footer/metadata rows if known (e.g., last 3 rows)
    # Adjust the number based on your data
    # Here, assuming last 3 rows are metadata
    df_cleaned = df_cleaned.iloc[:-3]  # Remove last 3 rows
    
    # Replace remaining NaN values if necessary
    # For numerical data, fill with 0
    df_cleaned.fillna(0, inplace=True)
    
    # Identify columns that should be numeric based on combined headers
    # This requires knowledge of the header naming; adjust accordingly
    # For example, columns containing 'increase' or 'decrease' are numeric
    numeric_cols = [col for col in df_cleaned.columns if 
                   'increase' in col.lower() or 'decrease' in col.lower()]
    
    # Convert identified columns to numeric types
    for col in numeric_cols:
        # Remove any non-numeric characters like commas or spaces
        df_cleaned[col] = df_cleaned[col].astype(str).str.replace(',', '').str.replace(' ', '')
        # Convert to numeric, coercing errors to NaN
        df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
        # Fill NaN values with 0 or appropriate value
        df_cleaned[col].fillna(0, inplace=True)
    
    # Add a 'Year' column based on the sheet name
    df_cleaned['Year'] = sheet
    
    # Store the cleaned DataFrame in the dictionary
    cleaned_data[sheet] = df_cleaned
    
    # Display a preview of the cleaned DataFrame
    print("Cleaned DataFrame Preview:")
    display(df_cleaned.head())


Available Sheets: ['2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998']

Processing Sheet: 2009
Combined Headers:
Column 0: ''
Column 1: 'Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes'
Column 2: ''
Column 3: 'Increase as a result of ...New buildings'
Column 4: ''
Column 5: 'Conversion'
Column 6: ''
Column 7: 'Decrease as a result of ...Demolition, fire etc.'
Column 8: ''
Column 9: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009



Processing Sheet: 2008
Combined Headers:
Column 0: ''
Column 1: 'Annual change in the number of residencesby type of activity and by municipality size class. 2008Municipalitysize classes'
Column 2: ''
Column 3: 'Increase as a result of ...New buildings'
Column 4: ''
Column 5: 'Conversion'
Column 6: ''
Column 7: 'Decrease as a result of ...Demolition, fire etc.'
Column 8: ''
Column 9: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


,Annual change in the number of residencesby type of activity and by municipality size class. 2008Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2558,114,828,1844,2008
1,"10,001 - 100,000 inhabitants",12063,471,683,11851,2008
2,"5,001 - 10,000 inhabitants",8767,401,412,8756,2008
3,"2,001 - 5,000 inhabitants",11614,1040,410,12244,2008
4,"2,000 inhabitants or fewer",9189,1085,252,10022,2008



Processing Sheet: 2007
Combined Headers:
Column 0: ''
Column 1: 'Annual change in the number of residencesby type of activity and by municipality size class. 2007Municipalitysize classes'
Column 2: ''
Column 3: 'Increase as a result of ...New buildings'
Column 4: ''
Column 5: 'Conversion'
Column 6: ''
Column 7: 'Decrease as a result of ...Demolition, fire etc.'
Column 8: ''
Column 9: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


...

,Annual change in the number of residencesby type of activity and by municipality size class. 2000Municipalitysize classes,Increase as a result of ...New buildings,Conversion 1),"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2149,-57,299,1793,2000
1,"10,001 - 100,000 inhabitants",7736,99,367,7468,2000
2,"5,001 - 10,000 inhabitants",6257,303,128,6432,2000
3,"2,001 - 5,000 inhabitants",8354,552,223,8683,2000
4,"2,000 inhabitants or fewer",7718,935,212,8441,2000



Processing Sheet: 1999
Combined Headers:
Column 0: 'Annual change in the number of residencesby type of activity and by municipality size class. 1999Municipalitysize classes'
Column 1: 'Increase as a result of ...New buildings'
Column 2: 'Conversion'
Column 3: 'Decrease as a result of ...Demolition, fire etc.'
Column 4: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


,Annual change in the number of residencesby type of activity and by municipality size class. 1999Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1416,102,341,1177,1999
1,"10,001 - 100,000 inhabitants",7639,229,367,7501,1999
2,"5,001 - 10,000 inhabitants",7009,269,148,7130,1999
3,"2,001 - 5,000 inhabitants",9027,735,243,9519,1999
4,"2,000 inhabitants or fewer",8017,1473,168,9322,1999



Processing Sheet: 1998
Combined Headers:
Column 0: 'Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes'
Column 1: 'Increase as a result of ...New buildings'
Column 2: 'Conversion'
Column 3: 'Decrease as a result of ...Demolition, fire etc.'
Column 4: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


,Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1818,78,370,1526,1998
1,"10,001 - 100,000 inhabitants",7887,319,429,7777,1998
2,"5,001 - 10,000 inhabitants",6539,337,159,6717,1998
3,"2,001 - 5,000 inhabitants",9215,644,241,9618,1998
4,"2,000 inhabitants or fewer",8275,1415,161,9529,1998


In [7]:
df_1998

,Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1818,78,370,1526,1998
1,"10,001 - 100,000 inhabitants",7887,319,429,7777,1998
2,"5,001 - 10,000 inhabitants",6539,337,159,6717,1998
3,"2,001 - 5,000 inhabitants",9215,644,241,9618,1998
4,"2,000 inhabitants or fewer",8275,1415,161,9529,1998
5,Agglomerations of the five largest cities,10502,440,565,10377,1998


In [40]:
df_1999 = cleaned_data['1999']
df_2000 = cleaned_data['2000']
df_2001 = cleaned_data['2001']
df_2002 = cleaned_data['2002']
df_2003 = cleaned_data['2003']
df_2004 = cleaned_data['2004']
df_2005 = cleaned_data['2005']
df_2006 = cleaned_data['2006']
df_2007 = cleaned_data['2007']
df_2008 = cleaned_data['2008']
df_2009 = cleaned_data['2009']

In [41]:
df_2009

,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009
5,Agglomerations of the five largest cities,12189,579,1778,10990,2009


In [42]:
df_2009 = df_2009.iloc[:-1].copy()
df_2008 = df_2009.iloc[:-1].copy()
df_2007 = df_2009.iloc[:-1].copy()
df_2006 = df_2009.iloc[:-1].copy()
df_2005 = df_2009.iloc[:-1].copy()
df_2004 = df_2009.iloc[:-1].copy()
df_2003 = df_2009.iloc[:-1].copy()
df_2002 = df_2009.iloc[:-1].copy()
df_2001 = df_2009.iloc[:-1].copy()
df_2000 = df_2009.iloc[:-1].copy()
df_1999 = df_1999.iloc[:-1].copy()
df_1998 = df_1998.iloc[:-1].copy()


In [49]:
df_1998

,Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1818,78,370,1526,1998
1,"10,001 - 100,000 inhabitants",7887,319,429,7777,1998
2,"5,001 - 10,000 inhabitants",6539,337,159,6717,1998


In [45]:
sum_row = {
    'Municipality size classes': 'Total',
    'Increase as a result of ...New buildings': sum_values['Increase as a result of ...New buildings'],
    'Conversion': sum_values['Conversion'],
    'Decrease as a result of ...Demolition, fire etc.': sum_values['Decrease as a result of ...Demolition, fire etc.'],
    'T 09.01.02Pure increase': sum_values['T 09.01.02Pure increase'],
    'Year': '2007'  
}

sum_values = df_2007[numerical_cols].sum()

In [46]:
sum_row_df = pd.DataFrame([sum_row])

In [47]:
df_2007 = pd.concat([df_2007, sum_row_df], ignore_index=True)

In [48]:
df_2007

,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year,Municipality size classes
0,"more than 100,000 inhabitants",2350,220,649,1921,2009,NaN
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009,NaN
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009,NaN
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009,NaN
4,NaN,31762,1703,3139,30326,2007,Total


In [19]:
sum_row_df

,Municipality size classes,Year,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase
0,sum_values,,56987,4652,4050,57589


In [20]:
df_07

NameError: name 'df_07' is not defined